In [1]:
import os, cv2, random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torch.nn.functional as F
from torch.utils import data


import torchvision.datasets as dset
import torchvision.transforms as T

In [2]:
TRAIN_DIR = './dataset/the-nature-conservancy-fisheries-monitoring/train/'
TEST_DIR = './dataset/the-nature-conservancy-fisheries-monitoring/test_stg1/'
FISH_CLASSES = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
ROWS = 128  
COLS = 128
CHANNELS = 3

### Loading and Preprocessing Data

In [3]:
# This part is from https://www.kaggle.com/jeffd23/deep-learning-in-the-deep-blue-lb-1-279

In [4]:
def get_images(fish):
    """Load files from train folder"""
    fish_dir = TRAIN_DIR+'{}'.format(fish)
    images = [fish+'/'+im for im in os.listdir(fish_dir)]
    return images

def read_image(src):
    """Read and resize individual images"""
    im = cv2.imread(src, cv2.IMREAD_COLOR)
    im = cv2.resize(im, (COLS, ROWS), interpolation=cv2.INTER_CUBIC)
    return im

In [5]:
files = []
y_all = []

for fish in FISH_CLASSES:
    fish_files = get_images(fish)
    files.extend(fish_files)
    
    y_fish = np.tile(fish, len(fish_files))
    y_all.extend(y_fish)
    print("{0} photos of {1}".format(len(fish_files), fish))
    
y_all = np.array(y_all)

1719 photos of ALB
200 photos of BET
117 photos of DOL
67 photos of LAG
465 photos of NoF
299 photos of OTHER
176 photos of SHARK
734 photos of YFT


In [6]:
y_all.shape

(3777,)

In [7]:
X_all = np.ndarray((len(files), ROWS, COLS, CHANNELS), dtype=np.uint8)

for i, im in enumerate(files): 
    X_all[i] = read_image(TRAIN_DIR+im)
    if i%1000 == 0: print('Processed {} of {}'.format(i, len(files)))

print(X_all.shape)

Processed 0 of 3777
Processed 1000 of 3777
Processed 2000 of 3777
Processed 3000 of 3777
(3777, 128, 128, 3)


In [8]:
X_all = X_all.transpose(0,3,1,2)

In [9]:
X_all.shape

(3777, 3, 128, 128)

### Splitting the Training Data

In [10]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

In [11]:
# One Hot Encoding Labels
y_all = LabelEncoder().fit_transform(y_all)
# y_all = to_categorical(y_all,8)

X_train, X_valid, y_train, y_valid = train_test_split(X_all, y_all, 
                                                    test_size=0.01, 
                                                    random_state=23, 
                                                    stratify=y_all)

In [12]:
y_train.shape

(3739,)

In [13]:
X_train = torch.from_numpy(X_train)
X_valid = torch.from_numpy(X_valid)
y_train = torch.from_numpy(y_train)
y_valid = torch.from_numpy(y_valid)

In [14]:
my_dataset_train = data.TensorDataset(X_train,y_train) # create your datset
loader_train = data.DataLoader(my_dataset_train,batch_size=50) # create your dataloader

In [15]:
my_dataset_val = data.TensorDataset(X_valid,y_valid) # create your datset
loader_val = data.DataLoader(my_dataset_val,batch_size=100) # create your dataloader

### Models

In [16]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
# print_every = 100

print('using device:', device)

using device: cpu


In [17]:
# Try add maxpool layer
model = None
optimizer = None

# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# pass

class ThreeLayerConvNet(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, channel_3, num_classes):
        super().__init__()

        #         pass
        self.cn1 = nn.Conv2d(in_channel, channel_1, kernel_size=5, stride=1, padding=2, bias=True)
        nn.init.kaiming_normal_(self.cn1.weight)
        
        self.cn2 = nn.Conv2d(channel_1, channel_2, kernel_size=5, stride=1, padding=2, bias=True)
        nn.init.kaiming_normal_(self.cn2.weight)
        
        self.cn3 = nn.Conv2d(channel_2, channel_3, kernel_size=5, stride=1, padding=2, bias=True)
        nn.init.kaiming_normal_(self.cn3.weight)
        
        self.fc = nn.Linear(channel_3*16*16, num_classes)
        nn.init.kaiming_normal_(self.fc.weight)
        
        self.batchnorm_conv1 = nn.BatchNorm2d(channel_1)
        self.batchnorm_conv2 = nn.BatchNorm2d(channel_2)
        self.batchnorm_conv3 = nn.BatchNorm2d(channel_3)
        
#         self.dropout1 = nn.Dropout2d(p=0.2)
#         self.dropout2 = nn.Dropout2d(p=0.2)
#         self.dropout3 = nn.Dropout2d(p=0.2)
        
        self.pool1 = nn.MaxPool2d((2, 2))
        self.pool2 = nn.MaxPool2d((2, 2))
        self.pool3 = nn.MaxPool2d((2, 2))
        
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        scores = None

        h1 = F.relu(self.cn1(x))
        
        h1_pool = self.pool1(h1)
        
        h1_pool = self.batchnorm_conv1(h1_pool)
        
        h2 = F.relu(self.cn2(h1_pool))
        
        h2_pool = self.pool2(h2)
        
        h2_pool = self.batchnorm_conv2(h2_pool)
        
        h3 = F.relu(self.cn3(h2_pool))
        
        h3_pool = self.pool3(h3)
        
        h3_pool = self.batchnorm_conv3(h3_pool)

        scores = self.fc(flatten(h3_pool))

        return self.softmax(scores)



In [18]:
print_every = 10
def train_part(loader_train,
               model, 
               optimizer, 
               epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
#     n_samples = x_train.shape[0]
    
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

        if e % print_every == 1:
            print('Iteration %d, loss = %.4f' % (t, loss.item()))
            check_accuracy_part(loader_val, model)
            check_accuracy_part(loader_train, model)
            print("")
    return model

In [19]:
def check_accuracy_part(loader,
                        model):
    """
    Check the accuracy of a classification model.
    
    Inputs:
    - model_fn: A function that performs the forward pass of the model,
      with the signature scores = model_fn(x, params)
    - params: List of PyTorch Tensors giving parameters of the model
    
    Returns: Nothing, but prints the accuracy of the model
    """
    print('Checking accuracy on the validation set')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [20]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

### Train the model

In [21]:
learning_rate =1e-2
model = ThreeLayerConvNet(in_channel=3, channel_1=32, channel_2=16, channel_3=8, num_classes=8)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.95)
final_model = train_part(loader_train, model, optimizer, epochs=50)

Iteration 74, loss = 1.8325
Checking accuracy on the validation set
Got 21 / 38 correct (55.26)
Checking accuracy on the validation set
Got 2089 / 3739 correct (55.87)

Iteration 74, loss = 1.3891
Checking accuracy on the validation set
Got 32 / 38 correct (84.21)
Checking accuracy on the validation set
Got 3084 / 3739 correct (82.48)

Iteration 74, loss = 1.2992
Checking accuracy on the validation set
Got 33 / 38 correct (86.84)
Checking accuracy on the validation set
Got 3311 / 3739 correct (88.55)

Iteration 74, loss = 1.2991
Checking accuracy on the validation set
Got 36 / 38 correct (94.74)
Checking accuracy on the validation set
Got 3450 / 3739 correct (92.27)

Iteration 74, loss = 1.2990
Checking accuracy on the validation set
Got 35 / 38 correct (92.11)
Checking accuracy on the validation set
Got 3516 / 3739 correct (94.04)



In [22]:
final_model

ThreeLayerConvNet(
  (cn1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (cn2): Conv2d(32, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (cn3): Conv2d(16, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc): Linear(in_features=2048, out_features=8, bias=True)
  (batchnorm_conv1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm_conv2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm_conv3): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (pool3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (softmax): Softmax(dim=1)
)

In [24]:
PATH = './model/cnn.pth'
torch.save(final_model.state_dict(), PATH)

In [26]:
final_model

ThreeLayerConvNet(
  (cn1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (cn2): Conv2d(32, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (cn3): Conv2d(16, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc): Linear(in_features=2048, out_features=8, bias=True)
  (batchnorm_conv1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm_conv2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm_conv3): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (pool3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (softmax): Softmax(dim=1)
)

### Output the result

In [27]:
test_files = [im for im in os.listdir(TEST_DIR)]
test = np.ndarray((len(test_files), ROWS, COLS, CHANNELS), dtype=np.uint8)

for i, im in enumerate(test_files): 
    test[i] = read_image(TEST_DIR+im)
    
# test_preds = model.predict(test, verbose=1)

In [28]:
test = test.transpose(0,3,1,2)

In [29]:
test.shape

(1000, 3, 128, 128)

In [30]:
test = torch.from_numpy(test)

In [31]:
test_preds = model(test.float())

In [32]:
test_preds

tensor([[5.1321e-01, 1.2301e-03, 2.3799e-03,  ..., 7.7977e-05, 2.9596e-01,
         3.3732e-03],
        [9.9990e-01, 9.2013e-11, 5.8428e-09,  ..., 1.0398e-09, 4.8902e-08,
         9.9583e-05],
        [9.9998e-01, 4.2699e-07, 3.6368e-08,  ..., 5.5728e-07, 1.7873e-05,
         8.0633e-08],
        ...,
        [7.0890e-03, 4.8651e-07, 2.0195e-06,  ..., 6.9889e-06, 1.0576e-04,
         2.7931e-06],
        [8.5088e-01, 1.6343e-04, 1.6566e-07,  ..., 2.4682e-05, 1.5261e-04,
         1.4845e-01],
        [9.9985e-01, 3.7505e-05, 5.9571e-07,  ..., 7.8504e-06, 8.7242e-10,
         2.0785e-06]], grad_fn=<SoftmaxBackward>)

In [33]:
FISH_CLASSES

['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

In [34]:
test_preds = test_preds.detach().numpy()

In [35]:
submission = pd.DataFrame(test_preds, columns=FISH_CLASSES)
submission.insert(0, 'image', test_files)
submission.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_02082.jpg,0.513207,1.230079e-03,2.379873e-03,2.242678e-04,1.835492e-01,7.797730e-05,2.959583e-01,3.373205e-03
1,img_02096.jpg,0.999900,9.201301e-11,5.842765e-09,3.926248e-09,9.943790e-09,1.039761e-09,4.890194e-08,9.958311e-05
2,img_00865.jpg,0.999981,4.269918e-07,3.636755e-08,2.895845e-10,2.451326e-08,5.572800e-07,1.787336e-05,8.063277e-08
3,img_01548.jpg,0.999948,2.382794e-05,1.964029e-05,1.475374e-06,4.159225e-06,4.894916e-08,2.790045e-06,2.372943e-09
4,img_06541.jpg,1.000000,7.185220e-09,1.789494e-10,2.836929e-12,8.936242e-11,4.981517e-10,9.579245e-13,3.438447e-11


In [36]:
submission = submission.set_index('image')

In [37]:
TEST_DIR_2 = './dataset/the-nature-conservancy-fisheries-monitoring/test_stg2/'

In [38]:
test_files_2 = [im for im in os.listdir(TEST_DIR_2)]

In [39]:
test_2 = np.ndarray((len(test_files_2), ROWS, COLS, CHANNELS), dtype=np.uint8)

In [40]:
for i, im in enumerate(test_files_2): 
    test_2[i] = read_image(TEST_DIR_2+im)

In [41]:
test_2 = test_2.transpose(0,3,1,2)
test_2 = torch.from_numpy(test_2)

In [ ]:
test_preds_2 = model(test_2.float())
test_preds_2 = test_preds_2.detach().numpy()

In [ ]:
submission2 = pd.DataFrame(test_preds_2, columns=FISH_CLASSES)

submission2.insert(0, 'image', test_files_2)
submission2.head()

In [ ]:
submission2 = submission2.set_index('image')

In [ ]:
submission2.shape

In [ ]:
final_submission = pd.concat([submission, submission2],axis=0)

In [ ]:
final_submission.to_csv("submission.csv")